In [ ]:
from werk_squad_tools import Zcalc_R23

In [ ]:
# just for now:

In [1]:
import Zcalc_R23

In [2]:
import numpy as np
import pandas as pd
import os
from os.path import expanduser
from glob import glob
from importlib import reload

In [3]:
reload(Zcalc_R23)

<module 'Zcalc_R23' from '/Users/bobzimmerman/Dropbox/IGM_spectra/jupyter_notebooks_SB/werksquadtools/werk-squad-tools/Zcalc_R23.py'>

In [4]:
home = expanduser('~')
path = os.path.join(home,'Dropbox','**','CGM2galaxies.csv')
filepath = glob(path, recursive=True)
vetrr = pd.read_csv(filepath[0])

In [5]:
# for testing the whole thing

m = vetrr['mstars']
OIII = vetrr['OIII_flux']
Hbeta = vetrr['Hbeta_flux']
NII = vetrr['NII_flux']
Ha = vetrr['Halpha_flux']
OII = vetrr['OII_flux']

CASES TESTED:

OII is -99: Check

good values but can't break degeneracy: Check

lower branch: Check

upper branch using mass: Check

N2 index: Check

N2O2 method: Check (there's actually only one like this in the data set, but whatever)

In [ ]:
# for testing individual cases

In [33]:
OII0, OIII0, NII0, Ha0, Hbeta0, m0 = 1e-16, 1e-17, 5e-18, 1.1e-17, 0.9e-18, 9.01

In [34]:
test = Zcalc_R23.findR23(OII0, OIII0, NII0, Ha0, Hbeta0, m0)

test

(3.0919006948630994, 'upper')

# Important discovery! The way I was doing it before did require a for loop when called.

Guess I'd better fix that.

In [8]:
def lower_branch(x,y):
    '''

    Function called when degeneracy is broken by galaxy mass of < 8 log(M_sun).

    Parameters
    ----------

    x : [float of some kind]
        log(R23) from findR23.
    y : [float of some kind]
        log(1.3*`OIII`/`OII`) from findR23.

    Returns
    -------

    Z : [float of some kind]
        Metallicity with units of (12 + log(O/H)).
    '''
    Z = 12.0 - 4.944 + 0.767*x + 0.602*x**2 - y*(0.29+0.332*x-0.331*x**2)

    return(Z)
def upper_branch(x,y):
    '''

    Called when degeneracy is broken by N2O2, N2 index, or mass > 9 log(M_sun).

    Parameters
    ----------

    x : [float of some kind]
        log(R23) from findR23.
    y : [float of some kind]
        log(1.3*`OIII`/`OII`) from findR23.

    Returns
    -------

    Z : [float of some kind]
        Metallicity with units of (12 + log(O/H)).
    '''
    Z = 12.0 - 2.939 - 0.2*x - 0.237*x**2 - 0.305*x**3 - 0.0283*x**4 - y*(
    0.0047 - 0.0221*x - 0.102*x**2 - 0.0817*x**3 - 0.00717*x**4)
    return(Z)


In [9]:
OIIHbeta = OII/Hbeta

OIIIHbeta = OIII/Hbeta

N2 = NII/Ha

O3N2 = np.log10(OIIIHbeta/N2)

N2O2 = NII/OII

R23 = OIIHbeta+OIIIHbeta

# actual arguments for metallicity functions
x = np.log10(R23)
y = np.log10(1.3*OIII/OII)

/Users/bobzimmerman/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
Z_R23 = np.empty([len(vetrr.z)])
flag = np.full([len(vetrr.z)], '', dtype='<U100')

# N2O2 method (if present at all):
if np.log10(N2O2).any() > -1.2:
    ind_N2O2 = np.ravel(np.where(np.log10(N2O2) > -1.2))
    Z_R23[ind_N2O2] = upper_branch(x[ind_N2O2],y[ind_N2O2])
    flag[ind_N2O2] = 'upper'
    
# N2 index:
N2_ind = np.ravel(np.where((N2 > 0.1)) and np.where((np.isnan(Z_R23) == True)))
Z_R23[N2_ind] = upper_branch(x[N2_ind],y[N2_ind])
flag[N2_ind] = 'upper'

# m:
mind_low = np.ravel(np.where(m < 8) and np.where((np.isnan(Z_R23) == True)))
Z_R23[mind_low] = lower_branch(x[mind_low],y[mind_low])
flag[mind_low] = 'lower'
        
mind_high = np.ravel(np.where(m > 9) and np.where((np.isnan(Z_R23) == True)))
Z_R23[mind_high] = upper_branch(x[mind_high],y[mind_high])
flag[mind_high] = 'upper w/ m'

# remains degenerate:
flag[np.where(np.isnan(Z_R23) == True)] = 'none'

# Okay great now back to testing.

In [11]:
import Zcalc_2

In [70]:
reload(Zcalc_2)

<module 'Zcalc_2' from '/Users/bobzimmerman/Dropbox/IGM_spectra/jupyter_notebooks_SB/werksquadtools/werk-squad-tools/Zcalc_2.py'>

In [62]:
OII0, OIII0, NII0, Ha0, Hbeta0, m0 = 1e-16, 1e-17, 5e-16, 1.1e-16, 0.9e-18, 9.01

In [63]:
test = Zcalc_R23.findR23(OII0, OIII0, NII0, Ha0, Hbeta0, m0)

test

(3.0919006948630994, 'upper')

In [71]:
test = Zcalc_2.findR23(OII0, OIII0, NII0, Ha0, Hbeta0, m0)

test

(array([3.09190069]), array(['upper'], dtype='<U100'))

In [65]:
# whole array now:

In [72]:
Zcalc_2.findR23(OII, OIII, NII, Ha, Hbeta, m)

(array([            nan,             nan,             nan,             nan,
         8.89733137e+00,             nan,             nan,  8.73394098e+00,
                    nan,  8.96861932e+00,             nan,  8.81790237e+00,
         8.79705079e+00,  8.66728882e+00,  8.79437491e+00,             nan,
         8.60512014e+00,             nan,  8.97230113e+00,  8.66178354e+00,
                    nan,             nan,  8.50752760e+00,             nan,
                    nan,  8.97230113e+00,             nan,  8.68240069e+00,
         8.74254732e+00,             nan,             nan,             nan,
                    nan,             nan,             nan,             nan,
         9.02171054e+00,             nan,             nan,  8.18003759e+00,
         8.76686831e+00,  8.97230113e+00,             nan,             nan,
         8.84764573e+00,  8.92936436e+00,             nan,             nan,
         8.76332432e+00,  8.68784052e+00,             nan,             nan,
            

In [68]:
# looks aight